In [ ]:
import pandas as pd
import numpy as np
import requests
import datetime
import os
import sys
sys.path.append('../..')

In [ ]:
from data.dataloader import Covid19IndiaLoader

In [ ]:
loader = Covid19IndiaLoader()
dataframes = loader.get_data()

In [ ]:
dataframes.keys()

In [ ]:
districtwise = dataframes['df_districtwise']
districtwise[districtwise['state'] == 'Delhi']['district'].unique()
# districtwise[districtwise['state'] == 'Gujarat']

In [ ]:
# dataframes['df_raw_data_2'].columns
# xyz = dataframes['df_raw_data_2']
# deceased = xyz[xyz['patientstatus'] == 'Deceased']
# # deceased[deceased['district'] == '']['state'].value_counts()
# state = deceased[deceased['state'] == 'Maharashtra']
# #  MANUAL CLEANUP + ASSUMPTIONS MADE IN CODE AFTER PRELIM MANUAL REVIEW
# state[state['district'] == ''].index[:20]
# state.loc[state[state['district'] == ''].index[:20]]
# # gj[gj['district'] == '']['district'] = 'Ahmadabad'

In [ ]:
def get_district_death_df(df_deaths_recoveries, state, district):
    deceased = df_deaths_recoveries[df_deaths_recoveries['patientstatus'] == 'Deceased']
    statedf = deceased[deceased['state'] == state]
    unknown = statedf[statedf['district'] == '']
    print(f'{len(unknown)} deaths in {state} with unknown district')
    if state == 'Gujarat':
        print(f'adding {len(unknown)} deaths to Ahmadabad count')
        statedf['district'][statedf['district'] == ''] = 'Ahmadabad'
        districtdf = statedf[statedf['district'].isin(district)]
    elif state == 'Maharashtra':
        num_unknown = len(unknown)
        thirds = num_unknown//3
        print(f'adding {thirds} deaths to Mumbai/Pune each')
        print(len(statedf[statedf['district'].isin(district)]))
        unknown_index = statedf['district'][statedf['district'] == ''].index
        statedf.loc[statedf[statedf['district'] == ''].index[:thirds]] = 'Mumbai'
        statedf.loc[statedf[statedf['district'] == ''].index[thirds:]] = 'Pune'
        print(len(statedf[statedf['district'].isin(district)]))
        districtdf = statedf[statedf['district'].isin(district)]
    elif state == 'Rajasthan':
        districtdf = statedf[statedf['district'].isin(district)]
    elif state == 'Karnataka':
        districtdf = statedf[statedf['district'].isin(district)]
    elif state == 'Delhi':
        num_unknown = len(unknown)
        print(f'adding {num_unknown} deaths to New Delhi')
        statedf['district'][statedf['district'] == ''] = 'New Delhi'
        districtdf = statedf[statedf['district'].isin(district)]
    return districtdf

In [ ]:
def get_district_time_series(dataframes, state='Karnataka', district='Bengaluru'):
    if district == 'all':
        districtwise = dataframes['df_districtwise']
        district = districtwise[districtwise['state'] == state]['district'].unique().tolist()
        district.append('')
    elif type(district) != list:
        district = [district]
        # district_timeseries = {}
        # for (s, d) in list(zip(state, district)):
        #     district_timeseries[d] = get_district_time_series(dataframes, state=s, district=d)
        # return district_timeseries
    # else:
    df_raw_data_1 = dataframes['df_raw_data'][dataframes['df_raw_data']['detectedstate'] == state]
    df_raw_data_1 = df_raw_data_1[df_raw_data_1['detecteddistrict'].isin(district)]
    if len(df_raw_data_1) == 0:
        return None
    df_raw_data_1['dateannounced'] = pd.to_datetime(df_raw_data_1['dateannounced'], format='%d/%m/%Y')
    index = pd.date_range(np.min(df_raw_data_1['dateannounced']), np.max(df_raw_data_1['dateannounced']))
    df_district = pd.DataFrame(columns=['total_confirmed'], index=index)
    df_district['total_confirmed'] = [0]*len(index)
    for _, row in df_raw_data_1.iterrows():
        df_district.loc[row['dateannounced']:, 'total_confirmed'] += 1

    # Deaths calculation
    deathsdf = get_district_death_df(dataframes['df_deaths_recoveries'], state, district)
    deathsdf = deathsdf[deathsdf['state'] == state]
    deathsdf = deathsdf[deathsdf['district'].isin(district)]
    deathsdf['date'] = pd.to_datetime(deathsdf['date'], format='%d/%m/%Y')
    df_district['total_deaths'] = [0]*len(index)
    
    for _, row in deathsdf.iterrows():
        if row['patientstatus'] == 'Deceased':
            date = pd.to_datetime(row['date'], format='%d/%m/%Y')
            df_district.loc[date:, 'total_deaths'] += 1

    df_district.reset_index(inplace=True)
    df_district.columns = ['date', 'total_confirmed', 'total_deaths']
    return df_district

In [ ]:
# df_raw_data_1
district_timeseries = get_district_time_series(dataframes, state='Delhi', district='all')

In [ ]:
district_timeseries

In [ ]:
districts = ['Mumbai', 'Bengaluru', 'Ahmadabad', 'Jaipur', 'Pune', 'New Delhi']
district_timeseries = get_district_time_series(dataframes, state=['Maharashtra', 'Karnataka', 'Gujarat', 'Rajasthan', 'Maharashtra', 'Delhi'], district=districts)
district_timeseries.keys()

In [ ]:
data = district_timeseries['Ahmadabad'].set_index('date')
data

In [ ]:
# get age data in bands
filenames = 'DDW-{}00C-13'
state_file_mapping = {
    filenames.format('24'): 'Gujarat',
    filenames.format('29'): 'Karnataka',
    filenames.format('27'): 'Maharashtra',
    filenames.format('07'): 'Delhi',
    filenames.format('08'): 'Rajasthan',
}

age_data = {}
directory = '../../data/data/census/'
for filename in os.listdir(directory):
    df = pd.read_excel(os.path.join(directory, filename))
    age_data[state_file_mapping[filename.split('.')[0]]] = df.dropna(how='all')

age_data.keys()

In [ ]:
def clean(raw_all_district_age_data):
    transposed = raw_all_district_age_data.head(3).T
    transposed.fillna('', inplace=True)
    new = transposed[transposed.columns[0]]
    for x in transposed.columns[1:]:
        new += transposed[x] 
    all_district_age_data = raw_all_district_age_data.copy()
    all_district_age_data.columns = new.T
    return all_district_age_data[4:]

In [ ]:
raw_all_district_age_data = age_data['Gujarat']


In [ ]:
all_district_age_data = clean(raw_all_district_age_data)

In [ ]:
test = clean(age_data['Karnataka'])
test['Area Name'].unique()

In [ ]:
# Get relevant district(s) data
district_age_data = all_district_age_data[all_district_age_data['Area Name'] == 'District - Ahmadabad (07)']
state_age_data = all_district_age_data[all_district_age_data['Area Name'] == 'State - GUJARAT (24)']
# or district_age_data['Distt.Code'] = 474 

In [ ]:
# district_age_data['Age'].unique()

In [ ]:
def create_age_bands():
    age_bands = {}
    for a in range(9):
        lower = a * 10
        upper = lower + 9
        if lower == 80:
            age_bands[f'{lower}+'] = list(range(lower, upper+11))
            age_bands[f'{lower}+'] += ['100+']
        else:
            age_bands[f'{lower}-{upper}'] = list(range(lower, upper+1))
    return age_bands

In [ ]:
age_bands = create_age_bands()
age_bands

In [ ]:
def get_age_band_population(age_data):
    # filter for age bands and sum
    total_pop = age_data[age_data['Age'] == 'All ages']['TotalPersons'].values[0]
    unknown_pop = age_data[age_data['Age'] == 'Age not stated']['TotalPersons'].values[0]
    total_pop -= unknown_pop

    age_band_pops = {}
    for key, band in age_bands.items():
        pop = age_data[age_data['Age'].isin(band)]['TotalPersons'].sum()
        age_band_pops[key] = pop
    return age_band_pops, total_pop

In [ ]:
state_age_band_pops, state_total_pop = get_age_band_population(state_age_data)
district_age_band_pops, district_total_pop = get_age_band_population(district_age_data)
district_age_band_pops, district_total_pop

In [ ]:
assert(state_total_pop == sum(state_age_band_pops.values()))
assert(district_total_pop == sum(district_age_band_pops.values()))

In [ ]:
state_age_band_ratios = {k: v / state_total_pop for k, v in state_age_band_pops.items()}
district_age_band_ratios = {k: v / district_total_pop for k, v in district_age_band_pops.items()}
district_age_band_ratios

In [ ]:
ref_age_band_ratios = {
    '0-9': 0.1144545912,
    '10-19': 0.1096780507,
    '20-29': 0.1387701325,
    '30-39': 0.1481915984,
    '40-49': 0.1548679659,
    '50-59': 0.1428622446,
    '60-69': 0.1092853481,
    '70-79': 0.05542319854,
    '80+': 0.02646687006,
}

# calculated here: https://docs.google.com/spreadsheets/d/1APX7XwoJPIbUXOgXa2vZNreDn6UseBucSGq9fh-N5jE/edit#gid=1859974541
ref_mortality_rate = {
    '0-9': 0.0002447933091,
    '10-19': 0.000377136454,
    '20-29': 0.0009389093188,
    '30-39': 0.001769966645,
    '40-49': 0.003645033792,
    '50-59': 0.01343325563,
    '60-69': 0.03884219978,
    '70-79': 0.08406020728,
    '80+': 0.1421208672,
}

In [ ]:
implied_mortality_age = {}
for k in ref_mortality_rate.keys():
    implied_mortality_age[k] = ref_age_band_ratios[k] * ref_mortality_rate[k]

implied_mortality = sum(implied_mortality_age.values())
# age weighted mortality based on other countries
implied_mortality

In [ ]:
# need mortality rate from India -- this is going to be dependent on case data, so introducing that testing bias...
daily_observed_mortality = data['total_deaths']/data['total_confirmed']
# observed deaths/known cases
daily_observed_mortality

In [ ]:
daily_mortality_ratio = daily_observed_mortality/implied_mortality
# how much different is it observed than 'implied'
daily_mortality_ratio

In [ ]:
age_stratified_daily_mortality = {}
for k in ref_mortality_rate.keys():
    age_stratified_daily_mortality[k] = daily_mortality_ratio * ref_mortality_rate[k]

# mortality rate accounting for observed/implied discrepancy
age_stratified_daily_mortality['70-79']

In [ ]:
age_std_mortality = sum([age_stratified_daily_mortality[k] * district_age_band_ratios[k] for k in district_age_band_ratios.keys()])
# mortality rate accounting for observed/implied discrepancy and weighted by model location age
age_std_mortality

In [ ]:
log_age_std_mortality = np.log(age_std_mortality)
log_age_std_mortality

In [ ]:
checker = pd.DataFrame()
print(district_total_pop)
checker['age_std'] = age_std_mortality
checker['non_std'] = data['total_deaths']/data['total_confirmed']
checker